In [1]:
import sys
import os
sys_path = os.path.abspath(os.path.join('..'))
import pandas as pd
from pathlib import Path

project_root = Path(os.path.abspath(os.path.join('..')))  
sys.path.insert(0, str(project_root))

from config import PROJECT_ROOT, DATA_RAW_DIR, SEASONS, START_YEAR, END_YEAR, TEST_SIZE, RANDOM_STATE

import plotly.express as px
import plotly.graph_objects as go

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np
# Load the dataset
df = pd.read_csv(DATA_RAW_DIR / f"all_races_{START_YEAR}_{END_YEAR}.csv")
# Display basic information about the dataset and checking datatypes
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1858 entries, 0 to 1857
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Position      1856 non-null   float64
 1   GridPosition  1856 non-null   float64
 2   FullName      1858 non-null   object 
 3   TeamName      1858 non-null   object 
 4   Status        1858 non-null   object 
 5   Points        1858 non-null   float64
 6   Year          1858 non-null   int64  
 7   Grand Prix    1858 non-null   object 
dtypes: float64(3), int64(1), object(4)
memory usage: 116.3+ KB


In [2]:
# Create a new DataFrame with only finished
df_finished = df[df['Status'] == 'Finished'].copy()
df_finished.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1214 entries, 0 to 1853
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Position      1214 non-null   float64
 1   GridPosition  1214 non-null   float64
 2   FullName      1214 non-null   object 
 3   TeamName      1214 non-null   object 
 4   Status        1214 non-null   object 
 5   Points        1214 non-null   float64
 6   Year          1214 non-null   int64  
 7   Grand Prix    1214 non-null   object 
dtypes: float64(3), int64(1), object(4)
memory usage: 85.4+ KB


In [3]:
# Scatter plot of GridPosition vs Position with trendline
fig = px.scatter(df_finished, x = "GridPosition", y = "Position", color = "TeamName", 
                 title = "GridPosition vs Position", labels = {"GridPosition": "Start",
                "Position": "Finish"}, trendline="ols")
fig.show()

In [4]:
# Histogram of Position Change (GridPosition - Position)
df_finished['PosChange'] = df_finished['GridPosition'] - df_finished['Position']

fig2 = px.histogram(df_finished, x = "PosChange", nbins=30, title = "Position Change Distribution", 
                    labels = {"PosChange": "Position Change (Grid - Finish)"})
fig2.show()

In [5]:
# Violin plot of Position Change
fig3 = px.violin(
    df_finished, 
    y='PosChange', 
    box=True,  # Shows box plot inside
    title='Violin plot change of positions (density distribution)',
    labels={'PosChange': 'Position Change'}
)
fig3.show()

In [6]:
# Box plot of Position Change by Team
fig4 = px.box(
    df_finished, 
    x='TeamName', 
    y='PosChange',
    color='TeamName',
    title='Change of positions by Team (2022-2025)',
    labels={'PosChange': 'Изменение позиции', 'TeamName': 'Команда'}
)
fig4.update_xaxes(tickangle=45)  # Rotate x-axis labels for better readability
fig4.show()

In [7]:
# Calculate and print median and mean of Position Change
median = df_finished['PosChange'].median()
mean = df_finished['PosChange'].mean()
print(median)
print(mean)

1.0
1.2825370675453047


In [8]:
# Choosing features and target variable for modeling
X = df_finished[['GridPosition']]
y = df_finished['Position']
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = TEST_SIZE, random_state = RANDOM_STATE)
# Display the sizes of the training and testing sets
print(f"Train size: {X_train.shape[0]}, Test size: {X_test.shape[0]}")


Train size: 971, Test size: 243


In [9]:
# Training a Linear Regression model
model_linearRegression = LinearRegression()
model_linearRegression.fit(X_train, y_train)

,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"tol tol: float, default=1e-6The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for the `lsqr` solver.`tol` is set as `atol` and `btol` of :func:`scipy.sparse.linalg.lsqr` whenfitting on sparse training data. This parameter has no effect when fittingon dense data... versionadded:: 1.7",1e-06
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation. This will only providespeedup in case of sufficiently large problems, that is if firstly`n_targets > 1` and secondly `X` is sparse or if `positive` is setto `True`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",None
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive. Thisoption is only supported for dense arrays.For a comparison between a linear regression model with positive constraintson the regression coefficients and a linear regression without such constraints,see :ref:`sphx_glr_auto_examples_linear_model_plot_nnls.py`... versionadded:: 0.24",False


In [10]:
# Making predictions and evaluating the model
y_pred = model_linearRegression.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")
print(f"R2 Score: {r2}")

Mean Absolute Error: 2.3336525793018787
R2 Score: 0.5466607286694428


In [11]:
# Linear Regression Line
x_range = np.linspace(df_finished['GridPosition'].min(), 
                      df_finished['GridPosition'].max(), 100)
y_range = model_linearRegression.predict(x_range.reshape(-1, 1))
# Plotting the results
fig.add_trace(go.Scatter(
    x=x_range, 
    y=y_range, 
    mode='lines',
    name=f'Регрессия (R²={r2:.3f})',
    line=dict(color='red', width=3)
))

# 3. Diagonal (ideal Grid=Position line)
fig.add_trace(go.Scatter(
    x=[1, 20], 
    y=[1, 20], 
    mode='lines',
    name='Grid = Position (no overtakes)',
    line=dict(color='green', width=2, dash='dash')
))

fig.update_layout(width=800, height=600)
fig.show()

c:\Users\User\Documents\trying_in_coding\pet-projects\F1\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

